# Manage Qiskit Serverless compute and data resources

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/serverless-manage-resources)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/serverless-manage-resources)
</Admonition>
</CloudContent>

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.1
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.0
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.8.1
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
scipy~=1.14.1
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.14.1
pyscf~=2.7.0
```
</details>

Qiskit Serverless allows you to manage compute and data across your [Qiskit pattern](./intro-to-patterns), including CPUs, QPUs, and other compute accelerators.

## Parallel workflows

For classical tasks that can be parallelized, use the `@distribute_task` decorator to define compute requirements needed to perform a task. Start by recalling the `transpile_remote.py` example from the [Write your first Qiskit Serverless program](./serverless-first-program) topic with the following code.

<LegacyContent>
The following code requires that you have already [saved your credentials](/guides/setup-channel#iqp).
</LegacyContent>
<CloudContent>
The following code requires that you have already [saved your credentials](/guides/cloud-setup).
</CloudContent>

In [1]:
# This cell is hidden from users, it just creates a new folder
from pathlib import Path

Path("./source_files").mkdir(exist_ok=True)

In [ ]:
%%writefile ./source_files/transpile_remote.py

from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_serverless import distribute_task

service = QiskitRuntimeService()

@distribute_task(target={"cpu": 1})
def transpile_remote(circuit, optimization_level, backend):
    """Transpiles an abstract circuit (or list of circuits) into an ISA circuit for a given backend."""
    pass_manager = generate_preset_pass_manager(
        optimization_level=optimization_level,
        backend=service.backend(backend)
    )
    isa_circuit = pass_manager.run(circuit)
    return isa_circuit

Writing ./source_files/transpile_remote.py


In this example, you decorated the `transpile_remote()` function with `@distribute_task(target={"cpu": 1})`. When run, this creates an asynchronous parallel worker task with a single CPU core, and returns with a reference to track the worker. To fetch the result, pass the reference to the `get()` function.

In [3]:
%%writefile --append ./source_files/transpile_remote.py

from qiskit_serverless import get, get_arguments, save_result

arguments = get_arguments()
circuit = arguments.get("circuit")
optimization_level = arguments.get("optimization_level")
backend = arguments.get("backend")

transpile_worker_reference = transpile_remote(
    circuit,
    optimization_level,
    backend
)
result = get(transpile_worker_reference)
save_result(result)

Appending to ./source_files/transpile_remote.py


You can also create and run multiple parallel tasks as follows.

In [4]:
%%writefile --append ./source_files/transpile_remote.py

transpile_worker_references = [
    transpile_remote(circuit, optimization_level, backend)
    for circuit in arguments.get("circuit_list")
]

results = get(transpile_worker_references)
save_result(results)  # Overwrites any previously saved results

Appending to ./source_files/transpile_remote.py


In [5]:
%%writefile --append ./source_files/transpile_remote.py
# This cell is hidden from users. It checks the transpilation ran correctly.
from qiskit import QuantumCircuit
assert all(isinstance(result, QuantumCircuit) for result in results)

Appending to ./source_files/transpile_remote.py


In [6]:
# This cell is hidden from users.
# It uploads the serverless program and checks it runs.


def test_serverless_job(title, entrypoint):
    # Import in function to stop them interfering with user-facing code
    from qiskit.circuit.random import random_circuit
    from qiskit_serverless import IBMServerlessClient, QiskitFunction
    import time
    import uuid

    title += "_" + uuid.uuid4().hex[:8]
    serverless = IBMServerlessClient()
    transpile_remote_demo = QiskitFunction(
        title=title,
        entrypoint=entrypoint,
        working_dir="./source_files/",
    )
    serverless.upload(transpile_remote_demo)
    job = serverless.get(title).run(
        circuit=random_circuit(3, 3),
        circuit_list=[random_circuit(3, 3) for _ in range(3)],
        backend="ibm_kyiv",
        optimization_level=1,
    )
    for retry in range(25):
        time.sleep(5)
        status = job.status()
        if status == "DONE":
            print("Job completed successfully")
            return
        if status not in ["QUEUED", "INITIALIZING", "RUNNING", "DONE"]:
            raise Exception(
                f"Unexpected job status '{status}'.\nHere's the logs:\n"
                + job.logs()
            )
        print(f"Waiting for job (status '{status}')")
    raise Exception("Job did not complete in time")


test_serverless_job(
    title="transpile_remote_serverless_test", entrypoint="transpile_remote.py"
)

Waiting for job (status 'QUEUED')


Waiting for job (status 'QUEUED')


Waiting for job (status 'QUEUED')


Waiting for job (status 'INITIALIZING')


Waiting for job (status 'RUNNING')


Waiting for job (status 'RUNNING')


Waiting for job (status 'RUNNING')


Waiting for job (status 'RUNNING')


Job completed successfully


### Explore different task configurations

You can flexibly allocate CPU, GPU, and memory for your tasks via `@distribute_task()`. For Qiskit Serverless on IBM Quantum&reg; Platform, each program is equipped with 16 CPU cores and 32 GB RAM, which can be allocated dynamically as needed.

CPU cores can be allocated as full CPU cores, or even fractional allocations, as shown in the following.

Memory is allocated in number of bytes. Recall that there are 1024 bytes in a kilobyte, 1024 kilobytes in a megabyte, and 1024 megabytes in a gigabyte. To allocate 2 GB of memory for your worker, you need to allocate `"mem": 2 * 1024 * 1024 * 1024`.

In [7]:
%%writefile --append ./source_files/transpile_remote.py

@distribute_task(target={
    "cpu": 16,
    "mem": 2 * 1024 * 1024 * 1024
})
def transpile_remote(circuit, optimization_level, backend):
    return None

Appending to ./source_files/transpile_remote.py


In [8]:
# This cell is hidden from users.
# It checks the distributed program works.
test_serverless_job(
    title="transpile_remote_serverless_test", entrypoint="transpile_remote.py"
)

Waiting for job (status 'QUEUED')


Waiting for job (status 'QUEUED')


Waiting for job (status 'QUEUED')


Waiting for job (status 'QUEUED')


Waiting for job (status 'QUEUED')


...

Waiting for job (status 'RUNNING')


Waiting for job (status 'RUNNING')


Waiting for job (status 'RUNNING')


Waiting for job (status 'RUNNING')


Job completed successfully


## Next steps

<Admonition type="info" title="Recommendations">

- See a full example that [ports existing code to Qiskit Serverless](./serverless-port-code).
- Read a paper in which researchers used Qiskit Serverless and quantum-centric supercomputing to [explore quantum chemistry](https://arxiv.org/abs/2405.05068v1).

</Admonition>